In [1]:
from seqdataset import *
import os
import pandas as pd
from tqdm import tqdm

In [2]:
pssm_path = '/Users/suhancho/data/Uniprot_metalbinding_challenge/merged_PSSM/'
pssm_files = [pssm_path+f for f in os.listdir(pssm_path)]
inpath = '/Users/suhancho/data/Uniprot_metalbinding_challenge/chebi/'
pos_fasta_path='/Users/suhancho/data/Uniprot_metalbinding_challenge/pos_sequence/'
neg_fasta_path='/Users/suhancho/data/Uniprot_metalbinding_challenge/neg_sequence/'

In [3]:
bind_tsv = process_binddata(inpath)

In [4]:
def getdata(pssm_in):
    pssm = get_PSSM_mat(pssm_in)
    proc_pssm = get_processed_pssm(pssm)
    pssm_accession = pssm_in.split('/')[-1].split('.')[0]
    if bind_tsv.Accession.tolist().count(pssm_accession):
        seq_type = 'positive'
        fasta_path = pos_fasta_path
    else : 
        seq_type = 'negative'
        fasta_path = neg_fasta_path    
    label = get_label(pssm_accession,fasta_path+pssm_accession+'.fasta',seq_type,bind_tsv)
    
    return(proc_pssm,label,pssm_accession)

In [5]:
pssm_outpath='/Users/suhancho/data/Uniprot_metalbinding_challenge/processed_pssm/'
label_outpath='/Users/suhancho/data/Uniprot_metalbinding_challenge/processed_label/'

In [6]:
# for pssm_in in tqdm(pssm_files):


def savedataset(pssm_in):
    df,label,pssm_accession = getdata(pssm_in)
    df.to_csv(pssm_outpath+pssm_accession+'.tsv',sep='\t')
    infile = open(label_outpath+pssm_accession+'.label.txt','w')
    label = [str(l) for l in label]
    infile.write(','.join(label)) ; infile.close()
    return(0)

In [10]:
import multiprocess
if __name__=='__main__':
    pool = multiprocess.Pool(processes=8)
    pool.map(savedataset,pssm_files)
    pool.close()
    pool.join()